In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import CountVectorizer
import pickle
import os, re
import nltk
BASE_DIR = '../input/'
LABELED_TRAIN_DF = BASE_DIR + 'labeled_train_clean_reviews_remove_stopwords.csv'
TEST_DF = BASE_DIR + 'test_clean_reviews_remove_stopwords.csv'

In [2]:
labeled_train = pd.read_csv(LABELED_TRAIN_DF, header = 0)
test = pd.read_csv(TEST_DF, header = 0)
labeled_train["review"] = labeled_train["review"].astype(str)
test["review"] = test["review"].astype(str)
print "Read %d labeled train reviews" \
          "and %d test reviews" % (labeled_train["review"].size, test["review"].size)

Read 25000 labeled train reviewsand 25000 test reviews


Data leakage

Check if test["sentiment"] is correct

In [3]:
test["sentiment"] = test["id"].map(lambda x: 1 if int(x.strip('"').split("_")[1]) >= 5 else 0)

Prepare

In [4]:
train_clean_reviews = labeled_train["review"].tolist()
test_clean_reviews = test["review"].tolist()

all_clean_reviews = train_clean_reviews + test_clean_reviews

In [5]:
print(np.random.choice(train_clean_reviews, 1))
print(np.random.choice(test_clean_reviews, 1))

['heard bad things cabin fever almost much heard cultish hype turns first film new impresario eli roth effort iq points dropping film progresses worse movies surely gory ones sure hype meter got high blood count hostel good amount genre fans premise necessarily bad either though kids go cabin week partying come across sick man covered blood panic set fire winds dead water feeds reservoir soon characters succumb flesh eating virus one way another characters either lead college kids including rider strong hero james dibello goofy side bar supporting village folks archetypal point inertia painfully meet fates townspeople get pretty weird seems non sensibly thrown together without many laughs pancakes kid comes nowhere maybe might funnier another movie context rest movie work also young police character even dumber less convincing others family goes dibello following incident possibilities realized roth pumps script common sense window sudden scares frights people hacking blood one another

In [6]:
%%script false
# We vectorize the text corpus by turning each text into a sequence of integers
# Each integer is the index of a token in the dictionary
from keras.preprocessing.text import Tokenizer

MAX_NUM_WORDS_FOR_KERAS_TOKENIZER = 500
#
# num_words: the maximum number of words to keep, based on frequency.
keras_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS_FOR_KERAS_TOKENIZER)
#
# fit_on_texts accepts a list of strings, a generator of strings or 
# a list of list of strings. In the last case, it assumes each entry of the lists to be a token.
# Here we provide a list of strings.
keras_tokenizer.fit_on_texts(train_clean_reviews)
word_index = keras_tokenizer.word_index
print('Keras Tokenizer found %s unique tokens' % len(word_index))

In [7]:
%%script false
# each element is <type 'numpy.float64'> 
embedded_train = keras_tokenizer.texts_to_matrix(train_clean_reviews, mode='count')
embedded_test = keras_tokenizer.texts_to_matrix(test_clean_reviews, mode='count')

In [8]:
%%script false
print(len(embedded_train), len(embedded_train[0]))
print(len(embedded_test), len(embedded_test[0]))
print(embedded_train[0])
print(embedded_test[0])

In [9]:
%%script false
# Reshape data into a 25000 x 5000 x 1 array for feedint 1D Covolution
embedded_train = np.reshape(embedded_train, (len(embedded_train), MAX_NUM_WORDS_FOR_KERAS_TOKENIZER, 1))
embedded_test = np.reshape(embedded_test, (len(embedded_test), MAX_NUM_WORDS_FOR_KERAS_TOKENIZER, 1))

In [10]:
%%script false
print(len(embedded_train), len(embedded_train[0]), len(embedded_train[0][0]))
print(len(embedded_test), len(embedded_test[0]), len(embedded_test[0][0]))
print(embedded_train[0])
print(embedded_test[0])

In [11]:
%%script false
# Split train_sequences into train and validation. Ratio: 80/20
VALIDATION_SPLIT = 0.2
np.random.seed(1234)

# 
perm = np.random.permutation(len(embedded_train))
index_train = perm[:int(len(embedded_train)*(1-VALIDATION_SPLIT))]
index_val = perm[int(len(embedded_train)*(1-VALIDATION_SPLIT)):]

x_train = embedded_train[index_train]
x_val = embedded_train[index_val]
y_train = labeled_train["sentiment"][index_train].tolist()
y_val = labeled_train["sentiment"][index_val].tolist()

print('Randomly split %d pad sequences for training, %d for validation' % (len(x_train) ,len(x_val)))

In [12]:
%%script false
x_test = embedded_test
y_test = test["sentiment"]

In [13]:
MAX_FEATURES = 5000
# Bag of Words
vectorizer = CountVectorizer(analyzer = "word",   \
                         tokenizer = None,    \
                         preprocessor = None, \
                         stop_words = None,   \
                         max_features = MAX_FEATURES)

In [14]:
train_data_features = vectorizer.fit_transform(train_clean_reviews)
test_data_features = vectorizer.transform(test_clean_reviews)
train_data_features = train_data_features.toarray()
test_data_features = test_data_features.toarray()
train_data_features = [np.float64(e) for e in train_data_features]
test_data_features = [np.float64(e) for e in test_data_features]

In [15]:
print(len(train_data_features), len(train_data_features[0]))
print(len(test_data_features), len(test_data_features[0]))

(25000, 5000)
(25000, 5000)


In [16]:
# Reshape data into a 25000 x 5000 x 1 array for feedint 1D Covolution
train_data_features = np.reshape(train_data_features, (len(train_data_features), MAX_FEATURES, 1))
test_data_features = np.reshape(test_data_features, (len(test_data_features), MAX_FEATURES, 1))

In [17]:
print(len(train_data_features), len(train_data_features[0]), len(train_data_features[0][0]))
print(len(test_data_features), len(test_data_features[0]), len(test_data_features[0][0]))
print(train_data_features[0])
print(test_data_features[0])

(25000, 5000, 1)
(25000, 5000, 1)
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [0.]
 [0.]]


In [18]:
# train_data_features = np.expand_dims(train_data_features, axis=2)
# print(len(train_data_features), len(train_data_features[0]), len(train_data_features[0][0]))
# print(len(test_data_features), len(test_data_features[0]), len(test_data_features[0][0]))
# print(train_data_features[0][0])

In [19]:
# Split train_sequences into train and validation. Ratio: 80/20
VALIDATION_SPLIT = 0.2
np.random.seed(1234)

# 
perm = np.random.permutation(len(train_data_features))
index_train = perm[:int(len(train_data_features)*(1-VALIDATION_SPLIT))]
index_val = perm[int(len(train_data_features)*(1-VALIDATION_SPLIT)):]

x_train = train_data_features[index_train]
x_val = train_data_features[index_val]
y_train = labeled_train["sentiment"][index_train].tolist()
y_val = labeled_train["sentiment"][index_val].tolist()

print('Randomly split %d pad sequences for training, %d for validation' % (len(x_train) ,len(x_val)))

Randomly split 20000 pad sequences for training, 5000 for validation


In [20]:
x_test = test_data_features
y_test = test["sentiment"]

In [21]:
print(len(x_train), len(x_train[0]), len(x_train[0][0]))
print(len(x_val), len(x_val[0]), len(x_val[0][0]))
print(len(x_test), len(x_test[0]), len(x_test[0][0]))
print(len(y_train), len(y_val), len(y_test))

(20000, 5000, 1)
(5000, 5000, 1)
(25000, 5000, 1)
(20000, 5000, 25000)


## Model Architecture

In [22]:
from keras.models import Sequential
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Bidirectional, Conv1D, MaxPooling1D ,GlobalMaxPooling1D
from keras.initializers import Constant

Using TensorFlow backend.


In [23]:
def cnn_rnn_model(dropout_cnn=0.5, dropout_rnn=0.2, num_filters=64, kernel_size=2):
    model = Sequential()
    
    LSTM_UNITS = 32
    LSTM_DROPOUT = 0.
    LSTM_RECCURENT_DROPOUT = 0.

    output_layer = Dense(1, activation='sigmoid')
    
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, padding='valid', activation='relu', strides=1
                    , input_shape=(5000, 1)))
    # Cannot use GlobalMaxPooling since you're feeding it into RNN
    model.add(MaxPooling1D(2))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout_cnn))
    model.add(Bidirectional(LSTM(LSTM_UNITS, 
                                 dropout=LSTM_DROPOUT, 
                                 recurrent_dropout=LSTM_RECCURENT_DROPOUT
                                )))
    model.add(Dropout(dropout_rnn))
    model.add(output_layer)
    
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [24]:
model = cnn_rnn_model()

In [25]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 4999, 64)          192       
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 2499, 64)          0         
_________________________________________________________________
dense_2 (Dense)              (None, 2499, 32)          2080      
_________________________________________________________________
dropout_1 (Dropout)          (None, 2499, 32)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
Total para

In [26]:
# batch_size: number of samples per gradient update
history = model.fit(x_train, y_train, batch_size=64, epochs=1, validation_data=[x_val, y_val])

Train on 20000 samples, validate on 5000 samples
Epoch 1/1
20000/20000 [==============================] - 2778s 139ms/step - loss: 0.6871 - acc: 0.5328 - val_loss: 0.6733 - val_acc: 0.5888


In [27]:
model.save('cnn_bilstm_bow_1epoch.h5')

In [28]:
y_test_pred = model.predict(x_test)

KeyboardInterrupt: 

In [ ]:
y_test_pred_binary = map(lambda predict: 1 if predict > 0.5 else 0, y_test_pred)

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

In [ ]:
print("The AUC score for CNN-BiLSTM model is : %.5f." %roc_auc_score(y_test, y_test_pred_binary))

In [ ]:
# Write the test results
output = pd.DataFrame(data={"id": test["id"], "sentiment": y_test_pred_binary})
output.to_csv(os.path.join('../', 'output', "cnn_bilstm_bow_6epoch.csv"), index=False, quoting=3)
print "Wrote to cnn_bilstm_bow_6epoch.csv"

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)

plt.plot(epochs, loss, color='orange', label='Train')
plt.plot(epochs, val_loss, color='green', label='Validation')
plt.title('Loss: cnn_bilstm_bow_6epoch')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.savefig('../figures/cnn_bilstm_bow_6epoch_loss.png', dpi=900)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']

plt.plot(epochs, acc, color='orange', label='Train')
plt.plot(epochs, val_acc, color='blue', label='Validation')
plt.title('Accuracy: cnn_bilstm_bow_6epoch')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig('../figures/cnn_bilstm_bow_6epoch_accuracy.png', dpi=900)